In [ ]:
# importd
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import re
from pathlib import Path
from glob import glob
import pickle
import shutil
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole, MolsToGridImage
from rdkit.Chem import AllChem

In [ ]:
file = "../templates/core_dummy.sdf"
core = Chem.SDMolSupplier(file, removeHs=False, sanitize=False)
"""Mol: 
mol object of the Mo core with dummy atoms instead of ligands
"""
file_NH3 = "../templates/core_NH3_dummy.sdf"
core_NH3 = Chem.SDMolSupplier(file_NH3, removeHs=False, sanitize=False)

In [ ]:
core[0]

In [ ]:
mol = Chem.MolFromMolFile(
        "/home/magstr/Documents/schrock/diagrams_schrock/dft/cycle_restart/Mo_N2/ams.results/traj.mol", sanitize = False, removeHs=False)

# Initialize substructure string

# : matches aromatic bonds. Not == when it is delocalized
# '[c]:[c][N]'
# Smart for a nitrogen bound to aromatic carbon.
smart = '[c][N]'

patt = Chem.MolFromSmarts(smart)

# Get indice of the carbon atom
print(f'Has substructure match: {mol.HasSubstructMatch(patt)}')
indices = mol.GetSubstructMatches(patt)
print(f'Match at indices {indices}')

frag = Chem.FragmentOnBonds(mol, [117])
frags = Chem.GetMolFrags(frag,asMols=True,sanitizeFrags=False)


Chem.Draw.MolToImage(frags[0], size=(800, 400))

In [ ]:
m = Chem.MolFromSmiles(Chem.MolToSmiles(frags[0]))

In [ ]:
im = Draw.MolToImage(m,returnPNG=False)

In [ ]:
im.save('HIPT_image', format='png')


In [ ]:
Chem.Draw.MolToImage(mol, size=(800, 400))

In [ ]:
frag = Chem.FragmentOnBonds(mol, [117])

In [ ]:
Chem.Draw.MolToImage(frag, size=(800, 800))

In [ ]:
frags = Chem.GetMolFrags(frag,asMols=True,sanitizeFrags=False)

In [ ]:
frags[1]

In [ ]:
Chem.MolToSmiles(Chem.RemoveHs(frags[0]))

In [ ]:
Chem.MolFromSmiles('[NH2]c1cc(-c2c(C(C)C)cc(C(C)C)cc2C(C)C)cc(-c2c(C(C)C)cc(C(C)C)cc2C(C)C)c1')

In [ ]:
Chem.Draw.MolToImage(frags[0], size=(800, 800))

In [ ]:
Chem.MolToSmiles(Chem.RemoveHs(frags[0]))

In [ ]:
Draw.MolsToGridImage(frags)

In [ ]:
Chem.Draw.MolToImage(frags[1], size=(800, 800))

In [ ]:
# Load intermediate
mol = Chem.MolFromMolFile(
    "/home/magstr/Documents/nitrogenase/schrock/diagrams_schrock/dft/cycle_restart/Mo_N2/ams.results/traj.mol", sanitize = False, removeHs=True)
#mol.UpdatePropertyCache()
mol.GetAtomWithIdx(23).SetFormalCharge(1)

# Initialize substructure string

# : matches aromatic bonds. Not == when it is delocalized
# '[c]:[c][N]'
# Smart for a nitrogen bound to aromatic carbon.
smart = '[c][N]'

# Initialize pattern
patt = Chem.MolFromSmarts(smart)

# Substructure match
print(f'Has substructure match: {mol.HasSubstructMatch(patt)}')
indices = mol.GetSubstructMatches(patt)

# Visualize
#im = Chem.Draw.MolToImage(mol, size=(800, 800))
#im.show()
bonds=[]
# Cut the bonds between the nitrogen and the carbon.
for tuple in indices:
    # Get bond number
    bonds.append(mol.GetBondBetweenAtoms(tuple[0], tuple[1]).GetIdx())

# Cut
frag = Chem.FragmentOnBonds(mol, bonds,addDummies=True, dummyLabels=[(1, 1), (1,1), (1,1)])

#Draw cut
#Chem.Draw.MolToImage(frag, size=(800, 800))

# Split mol object into individual fragments. sanitizeFrags needs to be false, otherwise not work.
frags = Chem.GetMolFrags(frag, asMols=True, sanitizeFrags=False)

#final = Chem.RemoveHs(frags[1])
Chem.Draw.MolToImage(frags[1], size=(400, 200))



In [ ]:
Chem.Draw.MolToImage(frags[2])

In [ ]:
Chem.MolFromSmiles(Chem.MolToSmiles(Chem.RemoveHs(frags[2])))

In [ ]:
Chem.MolToSmiles(Chem.RemoveHs(frags[2]))

### Try to replace nitrogen with dummy instead

In [ ]:
# This doesnt work if valenve is not filled for N. Eg if not dummy atoms are added
#Draw.MolsToGridImage(frags)

In [ ]:
frags[1]

In [ ]:
# Print dummy
# Save frag to file
templates = '../templates/'
with open(templates+'core_N2.sdf', 'w+') as f:
    f.write(Chem.MolToMolBlock(frags[1]))


In [ ]:
# Convert dummy to hydrogen. Label possibly still persists
for a in frags[1].GetAtoms():
    if a.GetSymbol() == '*':
        a.SetAtomicNum(1)
templates = '../templates/'
        
# Save frag to file
with open(templates+'core_N2_NH3_hs.sdf', 'w+') as f:
    f.write(Chem.MolToMolBlock(frags[1]))

## Save frag to file
#with open('templates/HIPT.mol', 'w+') as f:
#    f.write(Chem.MolToMolBlock(frags[0]))

In [ ]:
# Replace dummy atoms
for a in frags[1].GetAtoms():
    if a.GetSymbol() == '*':
        a.SetAtomicNum(1)
        print('Dummy replaced')

In [ ]:
frags[1]

In [ ]:
with open(templates+'core_noHS.sdf', 'w+') as f:
    f.write(Chem.MolToMolBlock(frags[1]))

In [ ]:
# Alternative way to replace dummy? 
mod_mol = Chem.ReplaceSubstructs(frags[1],
                                 Chem.MolFromSmiles('[1*]'),
                                 Chem.MolFromSmiles('[H]'))
mod_mol[0]

In [ ]:
# Substructure match to find the fragment with the core
# Smart for a nitrogen bound to aromatic carbon.
smart = "[Mo]"

# Initialize pattern
patt = Chem.MolFromSmarts(smart)

# Substructure match
for idx,struct in enumerate(frags):
    if struct.HasSubstructMatch(patt):
        print(f"Found the molybdenum core: {idx}")
        break

## Mo_NH3 struct for GA template

In [ ]:
# Load intermediate
mol = Chem.MolFromMolFile(
    "../templates/core_NH3_N2.mol", sanitize = False, removeHs=False)

# Initialize substructure string

# : matches aromatic bonds. Not == when it is delocalized
# '[c]:[c][N]'
# Smart for a nitrogen bound to aromatic carbon.
smart = '[H][N][C]'

# Initialize pattern
patt = Chem.MolFromSmarts(smart)

# Substructure match
print(f'Has substructure match: {mol.HasSubstructMatch(patt)}')
indices = mol.GetSubstructMatches(patt)

# Visualize
#im = Chem.Draw.MolToImage(mol, size=(800, 800))
#im.show()
bonds=[]
# Cut the bonds between the nitrogen and the carbon.
for tuple in indices:
    # Get bond number
    bonds.append(mol.GetBondBetweenAtoms(tuple[0], tuple[1]).GetIdx())

# Cut
frag = Chem.FragmentOnBonds(mol, bonds,addDummies=True)

#Draw cut
#Chem.Draw.MolToImage(frag, size=(800, 800))

# Split mol object into individual fragments. sanitizeFrags needs to be false, otherwise not work.
frags = Chem.GetMolFrags(frag, asMols=True, sanitizeFrags=False)

Chem.Draw.MolToImage(frags[0], size=(400, 200))



In [ ]:
Chem.Draw.MolToImage(mol, size=(400, 200))

### Try to put N2 on side

In [ ]:
def getAttachmentVector(mol, atom_num=0):
    """Search for the position of the attachment point and extract the atom index of the attachment point and the connected atom (only single neighbour supported)
    Function from https://pschmidtke.github.io/blog/rdkit/3d-editor/2021/01/23/grafting-fragments.html
    mol: rdkit molecule with a dummy atom
    return: atom indices
    """
    rindex = -1
    rindexNeighbor = -1
    for atom in mol.GetAtoms():
        if atom.GetAtomicNum() == atom_num:
            rindex = atom.GetIdx()
            print(rindex)
            neighbours = atom.GetNeighbors()
            print(neighbours)
            if len(neighbours) == 1:
                rindexNeighbor = neighbours[0].GetIdx()
            else:
                print("two attachment points not supported yet")
                return None

    return rindex, rindexNeighbor

In [ ]:
def AddAtom(mol,indexNeighbor, atom_type="N"):
    """Replace an atom with another type"""

    emol = Chem.EditableMol(mol)
    idx = emol.AddAtom(Chem.Atom(atom_type))
    print(idx, indexNeighbor)
    emol.AddBond(idx, indexNeighbor, order=Chem.rdchem.BondType.SINGLE)
    
    secondN_idx = emol.AddAtom(Chem.Atom(atom_type))

    emol.AddBond(secondN_idx, idx, order=Chem.rdchem.BondType.TRIPLE)
    
    return emol.GetMol()

In [ ]:
mol2 = AddAtom(mol,7)

In [ ]:
mol2

In [ ]:
mol2.RemoveAllConformers()

In [ ]:
mol2

In [ ]:
mol2.GetAtomWithIdx(26).SetFormalCharge(1)
mol2.GetAtomWithIdx(30).SetFormalCharge(1)

In [ ]:
Chem.SanitizeMol(mol2)

In [ ]:
mol2

In [ ]:
from rdkit.Chem import AllChem

In [ ]:
cids = AllChem.EmbedMultipleConfs(
        mol=mol2,
        numConfs=1,
        maxAttempts=10,
        randomSeed=2,
    )

In [ ]:
len(cids)

In [ ]:
# Print dummy
# Save frag to file
templates = '../templates/'
with open(templates+'core_NH3_N2.mol', 'w+') as f:
    f.write(Chem.MolToMolBlock(mol2))

### Try to replace nitrogen with dummy instead

In [ ]:
# This doesnt work if valenve is not filled for N. Eg if not dummy atoms are added
#Draw.MolsToGridImage(frags)

In [ ]:
frags[0]

In [ ]:
# Convert dummy to hydrogen. Label possibly still persists
for a in frags[0].GetAtoms():
    if a.GetSymbol() == '*':
        a.SetAtomicNum(0)

In [ ]:
frags[0]

In [ ]:
# Print dummy
# Save frag to file
templates = '../templates/'
with open(templates+'core_NH3_N2_dummy.mol', 'w+') as f:
    f.write(Chem.MolToMolBlock(frags[0]))


In [ ]:
file_NH3 = "../templates/core_NH3_N2_dummy.mol"
core_NH3 = Chem.SDMolSupplier(file_NH3, removeHs=False, sanitize=False)

In [ ]:
core_NH3[0]

In [ ]:
Chem.MolFromSmiles('[N]([H])[H]')

In [ ]:
# Load intermediate
mol = Chem.MolFromMolFile(
    "../templates/core_NH3.mol", sanitize = False, removeHs=False)

# Initialize substructure string

# : matches aromatic bonds. Not == when it is delocalized
# '[c]:[c][N]'
# Smart for a nitrogen bound to aromatic carbon.
smart = '[H][N][C]'

# Initialize pattern
patt = Chem.MolFromSmarts(smart)

# Substructure match
print(f'Has substructure match: {mol.HasSubstructMatch(patt)}')
indices = mol.GetSubstructMatches(patt)

### Explore replace core functionality

In [ ]:
from rdkit.Chem import MolToSmiles, MolFromSmiles, ReplaceCore
mol = MolFromSmiles('CCN(CC)CC')
core = Chem.MolFromSmarts("[NX3;H0]")

In [ ]:
mol

In [ ]:
ReplaceCore(mol, core, mol.GetSubstructMatch(core))

In [ ]:
mol = Chem.MolFromSmiles('CC=C(C=CCCN1CCC(N)CC1)CCS')

In [ ]:
mol

In [ ]:
output_ligand = AllChem.ReplaceSubstructs(
            mol,
            Chem.MolFromSmarts("[c]"),
            Chem.MolFromSmarts("[H]"),
            replacementConnectionPoint=0,
            replaceAll=True
        )
res = Chem.MolFromSmiles(Chem.MolToSmiles(output_ligand))

In [ ]:
res

In [ ]:
output_ligand

In [ ]:
sys.path.insert(0, "../my_utils")
with open("/home/magstr/generation_data/prod_new5_0/GA27_debug.pkl", "rb") as f:
    gen = pickle.load(f)

In [ ]:
mol = gen.molecules[48].rdkit_mol
mol_with_atom_index(mol)
m = mol.GetSubstructMatch(Chem.MolFromSmarts("*C"))
